# Pre-requisites

Importing Packages, Function for fetching real time price of stock

In [60]:
import math
import numpy as np
import yfinance as yf
from scipy.stats import norm

In [61]:
# Fetch real-time stock price from Yahoo Finance
def real_time_price(symbol):
    stock = yf.Ticker(symbol)
    data = stock.history(period='1d', interval='1m')
    latest_price = data['Close'].iloc[-1]
    return latest_price

# Calculate historical volatility
def historical_volatility(symbol, period='1y'):
    stock = yf.Ticker(symbol)
    prices = stock.history(period=period)['Close']
    log_returns = np.log(prices / prices.shift(1)).dropna()
    volatility = np.std(log_returns) * np.sqrt(252)  # Annualizing the volatility
    return volatility

# Parameters

In [67]:
# User entered

symbol = 'TSLA'  # Tesla stock symbol
T = 2     # Time to expiration (in years)
K = 200   # Strike Price
N = 24    # For American Options

# Calculated

S0 = real_time_price(symbol)  # Current price of underlying asset
sigma = historical_volatility(symbol)
r = 0.05 # risk-free interest rate

print("Current price of {} is ${:.2f}".format(symbol, S0))
print("Current volatility of {} is ${:.2f}".format(symbol, sigma))
print("Strike price is ${} for an expiration period of {} years with risk-free interest rate of {}%".format(K, T, r))

Current price of TSLA is $197.57
Current volatility of TSLA is $0.50
Strike price is $200 for an expiration period of 2 years with risk-free interest rate of 0.05%


# European Options

Uses Black-Scholes model to generate Call/Put options pricing

In [68]:
def black_scholes(S0, K, sigma, r, T):

    # Calculate d1 and d2
    d1 = (np.log(S0 / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # Calculate call and put option prices
    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    return [call_price, put_price]
    
output = black_scholes(S0, K, sigma, r, T)

# Print option prices
print("Call price: ${:.3f}".format(output[0]))
print("Put price: ${:.3f}".format(output[1]))

Call price: $60.740
Put price: $44.138


# American Options

Uses Cox Ross Rubenstein (Binary Tree) model to generate Call/Put options pricing

In [69]:
def binom_tree(S0, K, sigma, r, T, N):
    # init
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    p = (np.exp(r*dt)-d)/(u-d)
    
    # price tree
    price_tree = np.zeros([N+1, N+1])
    
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j, i] = S0*(d**j)*(u**(i-j))
            
    # Options Value
    option_call = np.zeros([N+1, N+1])
    option_call[:, N] = np.maximum(np.zeros([N+1]), price_tree[:,N]-K)
    
    option_put = np.zeros([N+1, N+1])
    option_put[:, N] = np.maximum(np.zeros([N+1]), K-price_tree[:,N])
    
    # Option Value at t = 0
    for i in np.arange(N-1, -1, -1):
        for j in np.arange(0, i+1):
            option_call[j, i] = np.exp(-r*dt)*(p*option_call[j,i+1]+(1-p)*option_call[j+1, i+1])
            option_put[j, i] = np.exp(-r*dt)*(p*option_put[j,i+1]+(1-p)*option_put[j+1, i+1])
        
        
    # Return:
    return [option_call[0,0], option_put[0,0]]

output = binom_tree(S0, K, sigma, r, T, N)

# Print option prices
print("Call price: ${:.3f}".format(output[0]))
print("Put price: ${:.3f}".format(output[1]))

Call price: $60.364
Put price: $43.762
